In [103]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

try:
    import spacy
    SPACY_AVAILABLE = True
    print(" SpaCy available")
except ImportError:
    SPACY_AVAILABLE = False
    print(" SpaCy not available, will use NLTK fallback")
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer

    # nltk.download('punkt')
    # nltk.download('stopwords')
    # nltk.download('wordnet')

print(" All libraries imported successfully")

 SpaCy available
 All libraries imported successfully


In [104]:
CONFIG = {
    'data': {
        'file_path': "C:/Users/youssef/.cache/kagglehub/datasets/arshkon/linkedin-job-postings/versions/13/postings.csv",
        'max_it_jobs': 5000
    },
    'filtering': {
        'it_keywords': [
            # Programming & Development
            'software', 'developer', 'programmer', 'coding', 'programming',
            'frontend', 'backend', 'full stack', 'fullstack', 'web developer',
            'mobile developer', 'app developer', 'python', 'java', 'javascript',
            'react', 'angular', 'vue', 'node.js', 'django', 'flask',

            # Engineering & Architecture
            'software engineer', 'system engineer', 'solutions architect',
            'technical architect', 'platform engineer', 'site reliability',
            'devops', 'mlops', 'infrastructure engineer',

            # Data & Analytics
            'data scientist', 'data analyst', 'data engineer', 'machine learning',
            'artificial intelligence', 'ai engineer', 'ml engineer', 'deep learning',
            'big data', 'data mining', 'business intelligence', 'analytics',

            # Cybersecurity
            'cybersecurity', 'security analyst', 'information security',
            'penetration tester', 'security engineer', 'ethical hacker',

            # Infrastructure & Operations
            'system administrator', 'network engineer', 'cloud engineer',
            'aws', 'azure', 'google cloud', 'gcp', 'kubernetes', 'docker',
            'database administrator', 'dba', 'network administrator',

            # Quality & Testing
            'qa engineer', 'test engineer', 'automation tester', 'quality assurance',
            'software tester', 'manual tester',

            # IT Support & Management
            'it support', 'technical support', 'help desk', 'system analyst',
            'business analyst', 'technical analyst', 'it specialist'
        ],
        'min_description_length': 50
    },
    'preprocessing': {
        'min_token_length': 2,
        'max_tokens_per_job': 200
    },
    'encoding': {
        'max_cardinality_onehot': 10,
        'hash_features': 100
    }
}

print(" Configuration loaded")
print(f" Data path: {CONFIG['data']['file_path']}")
print(f" Target IT jobs: {CONFIG['data']['max_it_jobs']:,}")
print(f" IT keywords: {len(CONFIG['filtering']['it_keywords'])} keywords")

 Configuration loaded
 Data path: C:/Users/youssef/.cache/kagglehub/datasets/arshkon/linkedin-job-postings/versions/13/postings.csv
 Target IT jobs: 5,000
 IT keywords: 73 keywords


In [105]:
def load_data():
    print(" Loading dataset...")
    try:
        df = pd.read_csv(CONFIG['data']['file_path'])
        print(f"✓ Loaded {len(df):,} job postings")
        print(f"✓ Dataset shape: {df.shape}")
        print(f"✓ Columns: {list(df.columns)}")
        return df
    except FileNotFoundError:
        print("Please update the file path in CONFIG above")
        return None

df_original = load_data()
if df_original is not None:
    print(f"\n Dataset Info:")
    print(df_original.info())

 Loading dataset...
✓ Loaded 123,849 job postings
✓ Dataset shape: (123849, 31)
✓ Columns: ['job_id', 'company_name', 'title', 'description', 'max_salary', 'pay_period', 'location', 'company_id', 'views', 'med_salary', 'min_salary', 'formatted_work_type', 'applies', 'original_listed_time', 'remote_allowed', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type', 'normalized_salary', 'zip_code', 'fips']

 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 31 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   job_id                      123849 non-null  int64  
 1   company_name                122130 non-null  object 
 2   title                       123849 non-null  object 
 3   descript

In [106]:
def is_it_related(text, keywords):
    if pd.isna(text):
        return False

    text_lower = text.lower()
    return any(keyword in text_lower for keyword in keywords)

def filter_it_jobs(df):
    print("\n Filtering IT-related jobs...")
    print(f"Starting with {len(df):,} total jobs")

    it_keywords = CONFIG['filtering']['it_keywords']

    it_mask = (
        df['title'].apply(lambda x: is_it_related(x, it_keywords)) |
        df['description'].apply(lambda x: is_it_related(x, it_keywords))
    )

    it_jobs = df[it_mask].copy()
    print(f"✓ Found {len(it_jobs):,} IT-related jobs")

    if 'description' in it_jobs.columns:
        min_length = CONFIG['filtering']['min_description_length']
        before_filter = len(it_jobs)
        it_jobs = it_jobs[it_jobs['description'].str.len() >= min_length]
        print(f"✓ Removed {before_filter - len(it_jobs)} jobs with short descriptions")

    max_jobs = CONFIG['data']['max_it_jobs']
    if len(it_jobs) > max_jobs:
        print(f" Sampling {max_jobs:,} jobs from {len(it_jobs):,} available")
        it_jobs = it_jobs.sample(n=max_jobs, random_state=42)

    print(f" Final IT dataset: {len(it_jobs):,} jobs")
    return it_jobs

if df_original is not None:
    df_filtered = filter_it_jobs(df_original)

    del df_original

    print(f"\n Working with {len(df_filtered):,} IT jobs instead of 55K+ total jobs")
else:
    df_filtered = None


 Filtering IT-related jobs...
Starting with 123,849 total jobs
✓ Found 64,096 IT-related jobs
✓ Removed 6 jobs with short descriptions
 Sampling 5,000 jobs from 64,090 available
 Final IT dataset: 5,000 jobs

 Working with 5,000 IT jobs instead of 55K+ total jobs


In [107]:
def analyze_missing_data(df):
    print("\n Missing Data Analysis")
    print("-" * 40)

    missing_pct = df.isnull().mean() * 100
    missing_pct = missing_pct.sort_values(ascending=False)

    print("Missing data percentage by column:")
    for col, pct in missing_pct.items():
        if pct > 0:
            status = "🔴" if pct > 50 else "🟡" if pct > 20 else "🟢"
            print(f"  {status} {col}: {pct:.1f}%")

    return missing_pct

def clean_missing_data(df):
    print("\n Cleaning missing data...")

    missing_pct = analyze_missing_data(df)

    cols_to_drop = missing_pct[missing_pct > 80].index.tolist()
    if cols_to_drop:
        print(f"🗑  Dropping columns with >80% missing: {cols_to_drop}")
        df = df.drop(columns=cols_to_drop)

    essential_cols = ['job_id', 'company_name', 'title', 'description', 'location']

    before_rows = len(df)
    for col in essential_cols:
        if col in df.columns:
            df = df.dropna(subset=[col])

    after_rows = len(df)
    if before_rows != after_rows:
        print(f"  Removed {before_rows - after_rows:,} rows missing essential data")

    null_values = ['NA', 'na', 'NaN', 'nan', '', ' ']
    df = df.replace(null_values, np.nan)

    print(f" Cleaned dataset: {len(df):,} jobs remaining")
    return df


if df_filtered is not None:
    df_cleaned = clean_missing_data(df_filtered)
    print(f"\n Dataset reduced to {len(df_cleaned):,} clean IT jobs")
else:
    df_cleaned = None


 Cleaning missing data...

 Missing Data Analysis
----------------------------------------
Missing data percentage by column:
  🔴 closed_time: 99.0%
  🔴 skills_desc: 98.5%
  🔴 med_salary: 95.7%
  🔴 remote_allowed: 83.8%
  🔴 applies: 76.0%
  🔴 max_salary: 73.2%
  🔴 min_salary: 73.2%
  🔴 currency: 68.9%
  🔴 compensation_type: 68.9%
  🔴 pay_period: 68.9%
  🔴 normalized_salary: 68.9%
  🟡 posting_domain: 36.0%
  🟡 application_url: 31.7%
  🟡 fips: 24.6%
  🟡 formatted_experience_level: 23.1%
  🟢 zip_code: 19.2%
  🟢 views: 1.5%
  🟢 company_id: 1.2%
  🟢 company_name: 1.2%
🗑  Dropping columns with >80% missing: ['closed_time', 'skills_desc', 'med_salary', 'remote_allowed']
  Removed 59 rows missing essential data
 Cleaned dataset: 4,941 jobs remaining

 Dataset reduced to 4,941 clean IT jobs


In [108]:
nlp_model = None
stopwords_set = None
lemmatizer = None

def setup_text_processing():
    global nlp_model, stopwords_set, lemmatizer

    if SPACY_AVAILABLE:
        try:
            nlp_model = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
            print("✓ SpaCy model loaded successfully")
            return True
        except OSError:
            print("⚠ SpaCy model not found. Install with: python -m spacy download en_core_web_sm")

    try:
        stopwords_set = set(stopwords.words('english'))
        lemmatizer = WordNetLemmatizer()
        print("NLTK components loaded successfully")
        return False
    except:
        print("NLTK setup failed. Install required packages.")
        return None

use_spacy = setup_text_processing()
print(f" Text processing method: {'SpaCy' if use_spacy else 'NLTK'}")

✓ SpaCy model loaded successfully
 Text processing method: SpaCy


In [109]:
def process_text_spacy(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return []

    text = text.lower().strip()
    doc = nlp_model(text)

    tokens = []
    for token in doc:
        if (not token.is_stop and
            not token.is_punct and
            not token.is_space and
            len(token.text) >= CONFIG['preprocessing']['min_token_length'] and
            token.pos_ in ['NOUN', 'VERB', 'ADJ']):
            tokens.append(token.lemma_)

    max_tokens = CONFIG['preprocessing']['max_tokens_per_job']
    return tokens[:max_tokens]

def process_text_nltk(text):
    if not isinstance(text, str):
        return []

    import re

    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()

    if not text:
        return []

    tokens = word_tokenize(text)

    tokens = [token for token in tokens
              if token not in stopwords_set and
              len(token) >= CONFIG['preprocessing']['min_token_length']]

    if lemmatizer:
        tokens = [lemmatizer.lemmatize(token) for token in tokens]

    max_tokens = CONFIG['preprocessing']['max_tokens_per_job']
    return tokens[:max_tokens]

def process_descriptions(df):
    print("\n Processing job descriptions...")
    print(f"Processing {len(df):,} job descriptions...")

    if use_spacy and nlp_model is not None:
        print("Using SpaCy for text processing...")
        process_func = process_text_spacy
    else:
        print("Using NLTK for text processing...")
        process_func = process_text_nltk

    tqdm.pandas(desc="Processing descriptions")
    df['processed_tokens'] = df['description'].progress_apply(process_func)

    before_len = len(df)
    df = df[df['processed_tokens'].str.len() > 0].copy()
    after_len = len(df)

    if before_len != after_len:
        print(f"🗑  Removed {before_len - after_len} jobs with no meaningful tokens")

    print(f" Text processing complete: {len(df):,} jobs with processed descriptions")
    return df

if df_cleaned is not None:
    df_processed = process_descriptions(df_cleaned)
else:
    df_processed = None


 Processing job descriptions...
Processing 4,941 job descriptions...
Using SpaCy for text processing...








Processing descriptions:   0%|          | 0/4941 [00:00<?, ?it/s]





Processing descriptions:   0%|          | 2/4941 [00:00<05:20, 15.41it/s]





Processing descriptions:   0%|          | 4/4941 [00:00<06:06, 13.49it/s]





Processing descriptions:   0%|          | 6/4941 [00:00<07:04, 11.64it/s]





Processing descriptions:   0%|          | 8/4941 [00:00<07:59, 10.29it/s]





Processing descriptions:   0%|          | 11/4941 [00:00<06:00, 13.69it/s]





Processing descriptions:   0%|          | 13/4941 [00:00<05:41, 14.43it/s]





Processing descriptions:   0%|          | 15/4941 [00:01<06:04, 13.50it/s]





Processing descriptions:   0%|          | 18/4941 [00:01<05:16, 15.54it/s]





Processing descriptions:   0%|          | 20/4941 [00:01<06:02, 13.58it/s]





Processing descriptions:   0%|          | 22/4941 [00:01<06:25, 12.77it/s]





Processing descriptions:   1%|          | 25/4941 [00:01<05:54, 13.88it/s]





Processing descriptions:   1%|          | 27/49

 Text processing complete: 4,941 jobs with processed descriptions


In [110]:
def encode_categorical_features(df):
    print("\nEncoding categorical features...")

    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    text_cols = ['description', 'processed_tokens']
    categorical_cols = [col for col in categorical_cols if col not in text_cols]

    print(f"Encoding {len(categorical_cols)} categorical columns:")

    encoders = {}

    for col in categorical_cols:
        if col not in df.columns:
            continue

        cardinality = df[col].nunique()
        print(f"  {col}: {cardinality} unique values", end=" → ")

        if cardinality <= CONFIG['encoding']['max_cardinality_onehot']:
            dummies = pd.get_dummies(df[col], prefix=col, dummy_na=True)
            df = pd.concat([df, dummies], axis=1)
            df = df.drop(columns=[col])
            print("One-hot encoded")

        elif cardinality <= 1000:
            le = LabelEncoder()
            df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str))
            encoders[col] = le
            print("Label encoded")

        else:
            hasher = FeatureHasher(
                n_features=CONFIG['encoding']['hash_features'],
                input_type='string'
            )
            # Convert the column to a list of lists of strings
            hashed_input = [[str(val)] for val in df[col].astype(str)]
            hashed = hasher.fit_transform(hashed_input)

            hash_df = pd.DataFrame(
                hashed.toarray(),
                columns=[f'{col}_hash_{i}' for i in range(CONFIG['encoding']['hash_features'])],
                index=df.index
            )
            df = pd.concat([df, hash_df], axis=1)
            df = df.drop(columns=[col])
            print("Hash encoded")

    print(f"Feature encoding complete: {df.shape[1]} total features")
    return df, encoders

if df_processed is not None:
    df_encoded, feature_encoders = encode_categorical_features(df_processed)
    print(f"Final encoded dataset shape: {df_encoded.shape}")
else:
    df_encoded = None
    feature_encoders = {}


Encoding categorical features...
Encoding 13 categorical columns:
  company_name: 3025 unique values → Hash encoded
  title: 4214 unique values → Hash encoded
  pay_period: 3 unique values → One-hot encoded
  location: 1573 unique values → Hash encoded
  formatted_work_type: 7 unique values → One-hot encoded
  job_posting_url: 4941 unique values → Hash encoded
  application_url: 3375 unique values → Hash encoded
  application_type: 3 unique values → One-hot encoded
  formatted_experience_level: 6 unique values → One-hot encoded
  posting_domain: 1150 unique values → Hash encoded
  work_type: 7 unique values → One-hot encoded
  currency: 2 unique values → One-hot encoded
  compensation_type: 1 unique values → One-hot encoded
Feature encoding complete: 651 total features
Final encoded dataset shape: (4941, 651)


In [111]:
def validate_processed_data(df):
    print("\n Data Validation")
    print("-" * 30)

    issues = []

    print(f"Dataset shape: {df.shape}")
    print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    if 'processed_tokens' in df.columns:
        empty_tokens = (df['processed_tokens'].str.len() == 0).sum()
        if empty_tokens > 0:
            issues.append(f"{empty_tokens} rows have empty processed tokens")
        else:
            avg_tokens = df['processed_tokens'].str.len().mean()
            print(f"✓ Average tokens per job: {avg_tokens:.1f}")


    if 'title' in df.columns and 'company_name_encoded' in df.columns:
        duplicates = df.duplicated(subset=['title', 'company_name_encoded']).sum()
        if duplicates > 0:
            issues.append(f"{duplicates} potential duplicate job postings")
        else:
            print("✓ No duplicate job postings found")

    object_cols = df.select_dtypes(include=['object']).columns
    non_text_objects = [col for col in object_cols if col not in ['description', 'processed_tokens']]
    if non_text_objects:
        issues.append(f"Unencoded object columns: {non_text_objects}")
    else:
        print("✓ All categorical features properly encoded")

    missing_counts = df.isnull().sum()
    cols_with_missing = missing_counts[missing_counts > 0]
    if len(cols_with_missing) > 0:
        print("ℹ  Columns with missing values:")
        for col, count in cols_with_missing.items():
            print(f"   {col}: {count} ({count/len(df)*100:.1f}%)")

    if issues:
        print("\n⚠️  Issues found:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("\n All validation checks passed!")

    return issues



In [112]:
def save_processed_dataset(df, encoders, config):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    metadata_filename = f"processed_it_jobs_{timestamp}_metadata.json"

    print(f"\n Saving processed dataset...")

    metadata = {
        'processing_info': {
            'timestamp': str(datetime.now()),
            'total_jobs_processed': len(df),
            'original_dataset_size': 'Unknown (filtered early)',
            'processing_time_estimate': 'Fast (IT jobs filtered first)'
        },
        'dataset_info': {
            'shape': df.shape,
            'columns': df.columns.tolist(),
            'memory_usage_mb': df.memory_usage(deep=True).sum() / 1024**2
        },
        'configuration_used': config,
        'encoding_info': {
            'encoders_used': list(encoders.keys()),
            'encoding_methods': 'One-hot, Label, and Hash encoding based on cardinality'
        },
        'text_processing': {
            'method': 'SpaCy' if use_spacy else 'NLTK',
            'features': ['tokenization', 'stopword_removal', 'lemmatization', 'pos_filtering']
        },
        'data_quality': {
            'missing_data_handled': True,
            'duplicates_checked': True
        }
    }

    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2, default=str)

    print(f"✓ Metadata saved: {metadata_filename}")

    print(f"\n Sample of processed dataset:")
    display_cols = ['job_id', 'processed_tokens']
    if 'company_name' in df.columns:
        display_cols.insert(2, 'company_name')

    print(df[display_cols].head())

    print(f"\n All columns in final dataset ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i:2d}. {col}")

    return metadata_filename


if df_encoded is not None:
    saved_files = save_processed_dataset(df_encoded, feature_encoders, CONFIG)
    print(f"\n Processing completed successfully!")
    print(f" Files saved: {saved_files}")
else:
    print(" No data to save - check previous steps for errors")


 Saving processed dataset...
✓ Metadata saved: processed_it_jobs_20250928_161936_metadata.json

 Sample of processed dataset:
           job_id                                   processed_tokens
12660  3887701435  [representativearizona, wyoming, sale, timerem...
41442  3899523948  [role, digital, marketing, business, analystwo...
73490  3902932166  [apply, today, difference, mental, health, fie...
51728  3901391997  [requirement, description, requirement, global...
29389  3894292759  [consider, application, position, ongoing, bas...

 All columns in final dataset (651):
    1. job_id
    2. description
    3. max_salary
    4. company_id
    5. views
    6. min_salary
    7. applies
    8. original_listed_time
    9. expiry
   10. listed_time
   11. sponsored
   12. normalized_salary
   13. zip_code
   14. fips
   15. processed_tokens
   16. company_name_hash_0
   17. company_name_hash_1
   18. company_name_hash_2
   19. company_name_hash_3
   20. company_name_hash_4
   21. company_n

In [113]:
def analyze_it_dataset(df):
    """Comprehensive analysis of the processed IT dataset"""
    print("\n IT JOBS DATASET ANALYSIS")
    print("=" * 60)

    print(f" Dataset Overview:")
    print(f"   Total IT jobs: {len(df):,}")
    print(f"   Total features: {df.shape[1]:,}")

    company_col = 'company_name' if 'company_name' in df.columns else 'company_name_encoded'
    if company_col in df.columns:
        unique_companies = df[company_col].nunique()
        print(f"   Unique companies: {unique_companies:,}")

    location_col = 'location' if 'location' in df.columns else 'location_encoded'
    if location_col in df.columns:
        unique_locations = df[location_col].nunique()
        print(f"   Unique locations: {unique_locations:,}")

    if 'listed_time' in df.columns:
        try:
            dates = pd.to_datetime(df['listed_time'], unit='s')
            print(f"   Date range: {dates.min().date()} to {dates.max().date()}")
        except:
            print("   Date range: Unable to parse dates")

    print(f"\n💰 Salary Information:")
    salary_cols = ['max_salary', 'min_salary', 'normalized_salary']
    for col in salary_cols:
        if col in df.columns and not df[col].isna().all():
            available_count = df[col].notna().sum()
            available_pct = available_count / len(df) * 100
            print(f"   {col.replace('_', ' ').title()}:")
            print(f"      Available: {available_count:,} jobs ({available_pct:.1f}%)")
            if available_count > 0:
                print(f"      Range: ${df[col].min():,.0f} - ${df[col].max():,.0f}")
                print(f"      Median: ${df[col].median():,.0f}")

    if 'title' in df.columns:
        print(f"\n🏷️  Top 15 Job Titles:")
        top_titles = df['title'].value_counts().head(15)
        for i, (title, count) in enumerate(top_titles.items(), 1):
            print(f"   {i:2d}. {title}: {count}")


    if 'company_name' in df.columns:
        print(f"\n🏢 Top 15 Companies:")
        top_companies = df['company_name'].value_counts().head(15)
        for i, (company, count) in enumerate(top_companies.items(), 1):
            print(f"   {i:2d}. {company}: {count}")

    if 'processed_tokens' in df.columns:
        print(f"\n🛠️  Extracting top skills and keywords...")
        all_tokens = []
        for tokens in df['processed_tokens']:
            if isinstance(tokens, list):
                all_tokens.extend(tokens)

        if all_tokens:
            skill_freq = Counter(all_tokens)
            print(f"   Total unique tokens: {len(skill_freq):,}")
            print(f"   Total token occurrences: {len(all_tokens):,}")
            print(f"\n   Top 25 Skills/Keywords:")
            for i, (skill, count) in enumerate(skill_freq.most_common(25), 1):
                print(f"   {i:2d}. {skill}: {count}")

    work_type_cols = [col for col in df.columns if 'work_type' in col.lower()]
    if work_type_cols:
        print(f"\n💼 Work Type Distribution:")
        for col in work_type_cols:
            if df[col].sum() > 0:
                count = df[col].sum()
                pct = count / len(df) * 100
                clean_name = col.replace('formatted_work_type_', '').replace('work_type_', '')
                print(f"   {clean_name}: {count} ({pct:.1f}%)")

    exp_cols = [col for col in df.columns if 'experience_level' in col.lower()]
    if exp_cols:
        print(f"\n🎓 Experience Level Distribution:")
        for col in exp_cols:
            if df[col].sum() > 0:
                count = df[col].sum()
                pct = count / len(df) * 100
                clean_name = col.replace('formatted_experience_level_', '').replace('experience_level_', '')
                print(f"   {clean_name}: {count} ({pct:.1f}%)")

    return df


In [114]:
df_final = analyze_it_dataset(df_encoded)

print("\n" + "="*80)
print(" PREPROCESSING PIPELINE SUMMARY")
print("="*80)

if df_final is not None:
    print(f" Successfully processed {len(df_final):,} IT job postings")
    print(f" Final dataset: {df_final.shape[0]:,} rows × {df_final.shape[1]:,} columns")
    print(f" Memory usage: {df_final.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

    text_cols = len([col for col in df_final.columns if col in ['description', 'processed_tokens']])
    encoded_cols = len([col for col in df_final.columns if '_encoded' in col or '_hash_' in col])
    onehot_cols = len(df_final.columns) - text_cols - encoded_cols - len(['job_id', 'title', 'location', 'company_name']) if 'job_id' in df_final.columns else 0

    print(f"\n Feature Types:")
    print(f"   Text columns: {text_cols}")
    print(f"   Encoded features: {encoded_cols}")
    print(f"   One-hot features: {onehot_cols}")

else:
    print(" Processing failed - check previous cells for errors")

print("\n" + "="*80)


 IT JOBS DATASET ANALYSIS
 Dataset Overview:
   Total IT jobs: 4,941
   Total features: 651
   Date range: Unable to parse dates

💰 Salary Information:
   Max Salary:
      Available: 1,317 jobs (26.7%)
      Range: $12 - $800,000
      Median: $87,318
   Min Salary:
      Available: 1,317 jobs (26.7%)
      Range: $10 - $500,000
      Median: $66,000
   Normalized Salary:
      Available: 1,530 jobs (31.0%)
      Range: $28 - $650,000
      Median: $88,400

🛠️  Extracting top skills and keywords...
   Total unique tokens: 40,889
   Total token occurrences: 916,208

   Top 25 Skills/Keywords:
    1. work: 9909
    2. team: 9300
    3. experience: 8201
    4. customer: 6808
    5. service: 6206
    6. include: 6049
    7. support: 5646
    8. business: 5275
    9. provide: 5224
   10. project: 4724
   11. management: 4682
   12. product: 4383
   13. ensure: 4308
   14. system: 4216
   15. company: 4194
   16. client: 3970
   17. year: 3910
   18. role: 3806
   19. skill: 3745
   20. de